In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
import regex
from datetime import datetime

%matplotlib inline

In [136]:
train_df =  pd.read_pickle('september_encoded_train_df.pickle')

In [76]:
sub_df = pd.read_csv('./data/sample_submit.csv')
store_info_df =  pd.read_csv('./data/store_info.csv')


In [77]:
import datetime


def fix_sample_sub(sub_df, store_info_df):
    def trans_date(day):
        daytime =datetime.datetime.strptime(day, '%Y-%m-%d')
        return daytime
    def sample_sub_date(sub_df):
        sub_df["date"] = sub_df["date"].apply(trans_date)

        return sub_df
    def get_week_day(sub_df):
        def sun_to_mon(num):
            num = num.weekday()
            num -= 1
            return num%6
        sub_df["weekday"] = sub_df["date"].apply(sun_to_mon)
        return sub_df
        

    def merge_store_info(sub_df,store_info):
        sub_df = pd.merge(sub_df,store_info, on = "store_merchant_id", how = "left")

        return sub_df
        
    sub_df = sample_sub_date(sub_df)
    sub_df = get_week_day(sub_df)
    sub_df = merge_store_info(sub_df, store_info_df)
    return sub_df

sub_df = fix_sample_sub(sub_df, store_info_df)

In [158]:
def get_day(df):
    df["Year"]= df["date"].map(lambda x: x.year)
    df["Month"]= df["date"].map(lambda x: x.month)
    df["Day"]= df["date"].map(lambda x: x.day)
    return df
  
def kyuryobi(df):
    df["Payday"] = 0
    df["Payday"].loc[df["Day"].isin([10, 15, 25])]= 1
    return df

In [159]:
sub_df = get_day(sub_df)

In [160]:
sub_df

,date,store_merchant_id,transaction,weekday,merchant_id,store_id,category_name,sub_category_name,prefecture,city,...,humidity,rain,weather,prefecture_area,store_merchant_id_count,sub_category_name_count,holiday_flg,Year,Month,Day
0,2019-09-30,0,NaN,5,350,0,2,11,1,566,...,6,0,2,12,NaN,0.121553,0,2019,9,30
1,2019-09-30,1,NaN,5,3,1,2,17,10,50,...,7,0,1,2,NaN,0.009507,0,2019,9,30
2,2019-09-30,2,NaN,5,65,2,2,4,9,167,...,5,2,1,8,0.000019,0.606838,0,2019,9,30
3,2019-09-30,3,NaN,5,564,3,2,26,5,237,...,0,0,3,0,0.000019,0.032534,0,2019,9,30
4,2019-09-30,4,NaN,5,179,4,2,24,9,683,...,5,2,1,8,0.000019,0.010125,0,2019,9,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51849,2019-09-30,51849,NaN,5,340,49592,4,27,11,451,...,8,0,1,3,NaN,0.028175,0,2019,9,30
51850,2019-09-30,51850,NaN,5,477,49593,1,5,6,85,...,10,0,0,10,NaN,0.017299,0,2019,9,30
51851,2019-09-30,51851,NaN,5,534,49594,2,4,4,174,...,4,0,1,4,NaN,0.606838,0,2019,9,30
51852,2019-09-30,51852,NaN,5,187,49595,2,4,10,468,...,7,0,1,2,NaN,0.606838,0,2019,9,30


In [180]:
def get_weather(train_df):
    prefecture_name_df=pd.read_csv('./data/weather_data2/prefecture_name.csv')
    prefecture_names =  prefecture_name_df['Unnamed: 1']

    import csv
    with open('./data/weather_data2/weather_daily/aichi_daily.csv') as f:
        reader = csv.reader(f)
        next(reader)
        print(next(reader))

    import csv
    prefecture_weather_dict = dict()
    for fname in prefecture_names:
        with open('./data/weather_data2/weather_daily/' + fname + '_daily.csv') as f:
            reader = csv.reader(f)
            next(reader)
            weather_dict = dict()
            for _, date,temperature, humidity, rain, weather in reader:
                weather_dict[date] = (temperature, humidity, rain, weather)
                
        prefecture_weather_dict[fname] = weather_dict
    

    weather_dict = {fname: pd.read_csv('./data/weather_data2/weather_daily/' + fname + '_daily.csv') for fname in prefecture_names}

    # train_df['prefecture_r'] = prefecture_name_df[prefecture_name_df[train_df['prefecture']]]

    for kanji, romaji in prefecture_name_df.values:
        print(kanji, romaji)
        train_df[train_df.prefecture == kanji]['prefecture'] = romaji

        
    pre_dict = {kanji: romaji for kanji, romaji in prefecture_name_df.values}
    train_df['prefecture_r'] = train_df.prefecture.map(
        lambda x: pre_dict[x]
    )

    #train_df.to_pickle('r_prefecture_added.csv')
    print(type(weather_dict))
    weather_dict_dict = {key: value.to_dict() for key, value in weather_dict.items()}
    
    

    from datetime import datetime
    #prefecture_weather_dict_2 = dict()
    for key, value in prefecture_weather_dict.items():
        new_dict = dict()
        old_dict = value
        for key_in, value_in in old_dict.items():
            new_dict[datetime.strptime(key_in, '%Y-%m-%d')] = old_dict[key_in]
        prefecture_weather_dict[key] = new_dict
    def map_function(prefecture_r, date):
        try:
            return  prefecture_weather_dict[prefecture_r][date]
        except:
            print(prefecture_r, date)
            raise

    temp = train_df.apply(lambda x: map_function(x['prefecture_r'], x['date']), axis=1)
    train_df['temperature'] = temp.map(lambda x: x[0])
    train_df['humidity'] = temp.map(lambda x: x[1])
    train_df['rain'] = temp.map(lambda x: x[2])
    train_df['weather'] = temp.map(lambda x: x[3])
    
    
    
    return train_df

,date,store_merchant_id,transaction,weekday,merchant_id,store_id,category_name,sub_category_name,prefecture,city,village_section,scan,prefecture_r,temperature,humidity,rain,weather
0,2019-09-30,0000914aba4b71d1ab089515494eaf7548d266afb20edb...,NaN,5,9972cc88e494b1a81168e8932da381e5ca01d2a14dc0a0...,0000914aba4b71d1ab089515494eaf7548d266afb20edb...,各種小売,医薬品・化粧品・ドラッグストア,兵庫県,神戸市,三宮町,U_Scan,hyogo,27.6375,69.79166666666667,0.0,3.0
1,2019-09-30,0000f2c81eda0f4575de8dd14186913fb746a7f0693c2e...,NaN,5,016f7069d87c6718b2ea03b4692936208faa3a03682a87...,0000f2c81eda0f4575de8dd14186913fb746a7f0693c2e...,各種小売,書籍・文具・楽器・スポーツ・自転車・おもちゃ・CD・DVD,東京都,世田谷区,太子堂,U_Scan,tokyo,25.183333333333334,70.04166666666667,0.0,2.0
2,2019-09-30,00022d2e137ee4fccd3bf09d727cbd3f09404bea1dcba7...,NaN,5,1a9513f7b34d9a334e042c1718a48229b08a6c7b10b249...,00022d2e137ee4fccd3bf09d727cbd3f09404bea1dcba7...,各種小売,コンビニ,愛知県,名古屋市,築盛町,M_Scan,aichi,27.112499999999997,69.54166666666667,1.5,2.0
3,2019-09-30,000484f9b96841b4403d373e007624f47cfa9c29005e40...,NaN,5,fce91f4f54d52b3e35ef058cb1523f4345564b398e699c...,000484f9b96841b4403d373e007624f47cfa9c29005e40...,各種小売,食料品・酒類,大阪府,大阪市,阿波座,U_Scan,osaka,27.73333333333333,64.66666666666667,0.0,4.0
4,2019-09-30,0005ee0a8000b96b03f814a7208c5bc6616f753caf7e87...,NaN,5,4ac969a0bd9ca2eda70f52b04ddde5011787c8257bc32c...,0005ee0a8000b96b03f814a7208c5bc6616f753caf7e87...,各種小売,食品スーパー,愛知県,豊橋市,牛川通,U_Scan,aichi,27.112499999999997,69.54166666666667,1.5,2.0


In [129]:
sub_df = get_weather(sub_df)

['0', '2019-7-1', '24.308333333333326', '88.83333333333333', '10.5', '10.0']
北海道 sapporo
青森県 aomori
岩手県 iwate
秋田県 akita
宮城県 miyagi
山形県 yamagata
福島県 fukushima
茨城県 ibaragi
千葉県 chiba
栃木県 tochigi
埼玉県 saitama
東京都 tokyo
群馬県 gunma
山梨県 yamanashi
神奈川県 kanagawa
新潟県 nigata
長野県 nagano
静岡県 shizuoka
富山県 toyama
岐阜県 gifu


/home/ec2-user/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


愛知県 aichi
石川県 ishikawa
福井県 fukui
滋賀県 shiga
三重県 mie
奈良県 nara
和歌山県 wakayama
京都府 kyoto
大阪府 osaka
兵庫県 hyogo
鳥取県 tottori
島根県 shimane
岡山県 okayama
広島県 hiroshima
山口県 yamaguchi
香川県 kagawa
愛媛県 ehime
徳島県 tokushima
高知県 kochi
福岡県 fukuoka
佐賀県 saga
長崎県 nagasaki
大分県 oita
熊本県 kumamoto
宮崎県 miyazaki
鹿児島県 kagoshima
沖縄県 okinawa
<class 'dict'>


In [131]:
area_dict = {
    '東京都': '2,187.42',
 '大阪府': '1,896.83',
 '神奈川県': '2,415.84',
 '愛知県': '5,164.06',
 '埼玉県': '3,797.25',
 '福岡県': '4,976.17',
 '北海道': '83,456.20',
 '千葉県': '5,156.58',
 '兵庫県': '8,395.47',
 '静岡県': '7,780.09',
 '茨城県': '6,095.69',
 '京都府': '4,613.00',
 '宮城県': '7,285.73',
 '広島県': '8,478.52',
 '岐阜県': '10,621.17'
}

def add_area(df):
    global area_dict
    df['prefecture_area'] = df.prefecture.map(lambda x: area_dict[x])
    
add_area(sub_df)

In [138]:
from sklearn.preprocessing import LabelEncoder

def target_enc(df, cols, target_col):
    for col in cols:
        for agg_type in ['mean','std']:
            new_col_name = col+target_col+agg_type
            temp_df = df[[col, 'TransactionAmt']]
            #temp_df['TransactionAmt'] = temp_df['TransactionAmt'].astype(int)
            temp_df = temp_df.groupby([col])[target_col].agg([agg_type]).reset_index().rename(
                                                    columns={agg_type: new_col_name})

            temp_df.index = list(temp_df[col])
            temp_df = temp_df[new_col_name].to_dict()   

            df[new_col_name] = df[col].map(temp_df)


def label_encode(df):
    for f in df.columns:
        if df[f].dtype.name =='object':
            
            lbl = LabelEncoder()
            lbl.fit(list(df[f].values.astype(str)))
            df[f] = lbl.transform(list(df[f].values))

    return df


In [139]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51854 entries, 0 to 51853
Data columns (total 18 columns):
date                 51854 non-null datetime64[ns]
store_merchant_id    51854 non-null object
transaction          0 non-null float64
weekday              51854 non-null int64
merchant_id          51854 non-null object
store_id             51854 non-null object
category_name        51854 non-null object
sub_category_name    51854 non-null object
prefecture           51854 non-null object
city                 50849 non-null object
village_section      50468 non-null object
scan                 51854 non-null object
prefecture_r         51854 non-null object
temperature          51854 non-null object
humidity             51854 non-null object
rain                 51854 non-null object
weather              51854 non-null object
prefecture_area      51854 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(15)
memory usage: 7.5+ MB


In [140]:
label_encode(sub_df)

,date,store_merchant_id,transaction,weekday,merchant_id,store_id,category_name,sub_category_name,prefecture,city,village_section,scan,prefecture_r,temperature,humidity,rain,weather,prefecture_area
0,2019-09-30,0,NaN,5,350,0,2,11,1,566,345,1,5,13,6,0,2,12
1,2019-09-30,1,NaN,5,3,1,2,17,10,50,5159,1,14,6,7,0,1,2
2,2019-09-30,2,NaN,5,65,2,2,4,9,167,10955,0,0,11,5,2,1,8
3,2019-09-30,3,NaN,5,564,3,2,26,5,237,13215,1,10,14,0,0,3,0
4,2019-09-30,4,NaN,5,179,4,2,24,9,683,9903,1,0,11,5,2,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51849,2019-09-30,51849,NaN,5,340,49592,4,27,11,451,1720,0,7,7,8,0,1,3
51850,2019-09-30,51850,NaN,5,477,49593,1,5,6,85,3132,1,9,1,10,0,0,10
51851,2019-09-30,51851,NaN,5,534,49594,2,4,4,174,2799,0,11,3,4,0,1,4
51852,2019-09-30,51852,NaN,5,187,49595,2,4,10,468,3014,0,14,6,7,0,1,2


In [166]:
train_df , sub_df = count_encoding(train_df,'sub_category_name', sub_df ) 
#train_df , sub_df = count_encoding(train_df,'city', sub_df ) 
#train_df , sub_df = count_encoding(train_df,'village_section', sub_df ) 


KeyError: 'city'

In [155]:
sub_df["holiday_flg"] = 0

In [164]:
sub_df.columns

Index(['date', 'store_merchant_id', 'transaction', 'weekday', 'merchant_id',
       'store_id', 'category_name', 'sub_category_name', 'prefecture', 'city',
       'village_section', 'scan', 'prefecture_r', 'temperature', 'humidity',
       'rain', 'weather', 'prefecture_area', 'store_merchant_id_count',
       'sub_category_name_count', 'holiday_flg', 'Year', 'Month', 'Day'],
      dtype='object')

In [165]:
train_df.columns

Index(['weekday', 'holiday_flg', 'transaction', 'merchant_id', 'store_id',
       'store_merchant_id', 'active_user', 'GMV', 'category_name',
       'sub_category_name', 'prefecture', 'scan', 'weather', 'humidity',
       'rain', 'Month', 'Day', 'date', 'prefecture_area',
       'store_merchant_id_count', 'sub_category_name_count'],
      dtype='object')

In [167]:
use_columns= ['weekday', 'holiday_flg', 'transaction', 'category_name',
       'sub_category_name', 'prefecture', 'scan', 'weather', 'humidity',
       'rain', 'Month', 'Day', 'prefecture_area', 'sub_category_name_count', ]

In [181]:
sub_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51854 entries, 0 to 51853
Data columns (total 24 columns):
date                       51854 non-null datetime64[ns]
store_merchant_id          51854 non-null int64
transaction                0 non-null float64
weekday                    51854 non-null int64
merchant_id                51854 non-null int64
store_id                   51854 non-null int64
category_name              51854 non-null int64
sub_category_name          51854 non-null int64
prefecture                 51854 non-null int64
city                       51854 non-null int64
village_section            51854 non-null int64
scan                       51854 non-null int64
prefecture_r               51854 non-null int64
temperature                51854 non-null int64
humidity                   51854 non-null int64
rain                       51854 non-null int64
weather                    51854 non-null int64
prefecture_area            51854 non-null int64
store_merchant_id_co

In [202]:
def count_encoding(df, column ,test_df=None):
    encodeed = df.groupby(column)[column].count()/df.shape[0] #median or mean
    df[column +"_count"] = df[column].map(encodeed )
    if not(test_df is None):
        test_df[column +"_count"] = test_df[column].map(encodeed)
        return df, test_df
    return df


def get_group_window_mean(df, group_col ,target_col,  w_size = 2 ):
    new_col_name = target_col + "_group_win_mean"
    df_gb = df[[group_col, target_col]].copy()
    for_shift = df.groupby(group_col)[target_col].shift(1).isna()
    df_gb = df_gb.groupby(group_col)[target_col].rolling(w_size).mean().shift(1).reset_index().sort_values("level_1").reset_index(drop = True)
    df_gb.loc[for_shift, :] = np.nan
    df[new_col_name] = df_gb[target_col]
    return df
def get_group_window_std(df, group_col ,target_col,  w_size = 2 ):
    new_col_name = target_col + "_group_win_mean"
    df_gb = df[[group_col, target_col]].copy()
    for_shift = df.groupby(group_col)[target_col].shift(1).isna()
    df_gb = df_gb.groupby(group_col)[target_col].rolling(w_size).std().shift(1).reset_index().sort_values("level_1").reset_index(drop = True)
    df_gb.loc[for_shift, :] = np.nan
    df[new_col_name] = df_gb[target_col]
    return df

def target_enc(df, cols, target_col, test_df=None):
    for col in cols:
        for agg_type in ['mean', 'std']:
            new_col_name = col+target_col+agg_type
            temp_df = df[[col, target_col]]
            #temp_df['TransactionAmt'] = temp_df['TransactionAmt'].astype(int)
            temp_df = temp_df.groupby([col])[target_col].agg([agg_type]).reset_index().rename(
                                                    columns={agg_type: new_col_name})
            
            temp_df.index = list(temp_df[col])
            temp_df = temp_df[new_col_name].to_dict()   

            df[new_col_name] = df[col].map(temp_df)
                
            if not(test_df is None):
                test_df[col+target_col+agg_type] = test_df[col].map(temp_df)
    if not(test_df is None):
        return df, test_df
    else:  
        return df

In [192]:
train_df

,weekday,holiday_flg,transaction,merchant_id,store_id,store_merchant_id,active_user,GMV,category_name,sub_category_name,...,scan,weather,humidity,rain,Month,Day,date,prefecture_area,store_merchant_id_count,sub_category_name_count
4642582,0,1,13,505,35571,37141,13,18618,2,11,...,0,5,245,0,9,1,2019-09-01,2,0.000019,0.121553
4483342,0,1,5,534,18177,19020,5,2424,2,4,...,0,3,188,0,9,1,2019-09-01,11,0.000019,0.606838
4483343,0,1,5,216,1062,1115,5,3368,2,4,...,0,3,34,0,9,1,2019-09-01,14,0.000019,0.606838
3767929,0,1,23,534,34307,35829,23,9860,2,4,...,0,5,249,0,9,1,2019-09-01,12,0.000019,0.606838
3767928,0,1,23,534,17446,18259,22,14570,2,4,...,0,5,245,0,9,1,2019-09-01,2,0.000019,0.606838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863487,0,1,19,534,2272,2380,19,6597,2,4,...,0,5,323,0,9,29,2019-09-29,6,0.000019,0.606838
1863486,0,1,19,65,18087,18923,15,11096,2,4,...,0,5,121,0,9,29,2019-09-29,0,0.000019,0.606838
1863485,0,1,19,187,44005,45980,19,15310,2,4,...,0,5,123,0,9,29,2019-09-29,13,0.000019,0.606838
1863483,0,1,19,187,11124,11624,16,16407,2,4,...,0,5,323,0,9,29,2019-09-29,6,0.000019,0.606838


In [193]:
train_df = train_df.reset_index(drop = True)

In [183]:
import pandas as pd
import numpy as np
import os
import pdb

def get_recent_average(train_df, groupby, target, ntake=4):
    assert isinstance(train_df, pd.DataFrame)
    store_col_average = dict()

    entries = train_df[groupby].unique()
    for entry in entries:
        store_col_average[entry] = train_df[train_df[groupby] == entry][target].iloc[-ntake:].mean()
    return store_col_average


In [201]:
#targets = ['transaction']#,
targets = ['GMV' ,'active_user']
column = "store_merchant_id"

for target in targets:
    train_df = get_group_window_mean(train_df, "store_merchant_id" ,target,  w_size = 4 )
    ave_dict = get_recent_average(train_df, "store_merchant_id",target, ntake = 4 )
    sub_df[column +"_group_win_mean"] = sub_df[column].map(ave_dict)
    
    

In [203]:
train_df[train_df.store_merchant_id == 10]

,weekday,holiday_flg,transaction,merchant_id,store_id,store_merchant_id,active_user,GMV,category_name,sub_category_name,...,rain,Month,Day,date,prefecture_area,store_merchant_id_count,sub_category_name_count,transaction_group_win_mean,GMV_group_win_mean,active_user_group_win_mean
29731,0,1,8,216,10,10,8,6541,2,4,...,0,9,1,2019-09-01,14,0.000019,0.606838,NaN,NaN,NaN
62856,1,0,4,216,10,10,4,3602,2,4,...,30,9,2,2019-09-02,14,0.000019,0.606838,NaN,NaN,NaN
105651,2,0,3,216,10,10,3,2235,2,4,...,0,9,3,2019-09-03,14,0.000019,0.606838,NaN,NaN,NaN
163257,3,0,4,216,10,10,4,2973,2,4,...,0,9,4,2019-09-04,14,0.000019,0.606838,NaN,NaN,NaN
256477,4,0,6,216,10,10,6,2864,2,4,...,0,9,5,2019-09-05,14,0.000019,0.606838,4.75,3837.75,4.75
305662,5,0,6,216,10,10,6,2769,2,4,...,30,9,6,2019-09-06,14,0.000019,0.606838,4.25,2918.50,4.25
312972,6,1,9,216,10,10,7,7366,2,4,...,0,9,7,2019-09-07,14,0.000019,0.606838,4.75,2710.25,4.75
412675,0,1,3,216,10,10,3,836,2,4,...,0,9,8,2019-09-08,14,0.000019,0.606838,6.25,3993.00,5.75
443844,1,0,5,216,10,10,5,3280,2,4,...,0,9,9,2019-09-09,14,0.000019,0.606838,6.00,3458.75,5.50
471205,2,0,5,216,10,10,5,2297,2,4,...,0,9,10,2019-09-10,14,0.000019,0.606838,5.75,3562.75,5.25


In [204]:
sub_df[sub_df.store_merchant_id == 10]

,date,store_merchant_id,transaction,weekday,merchant_id,store_id,category_name,sub_category_name,prefecture,city,...,rain,weather,prefecture_area,store_merchant_id_count,sub_category_name_count,holiday_flg,Year,Month,Day,store_merchant_id_group_win_mean
10,2019-09-30,10,NaN,5,216,10,2,4,2,405,...,0,0,14,0.000019,0.606838,0,2019,9,30,7.25


In [211]:
train_df.to_pickle("./train_df_9_20_1.pickle")

In [212]:
sub_df.to_pickle("./sub_df_9_20_1.pickle")

In [190]:
column =  "store_merchant_id"
sub_df[column +"_group_win_mean"] = sub_df[column].map(ave_dict)

In [210]:
cols = ['store_merchant_id']
targets = ["transaction", 'GMV' ,'active_user']
for target in targets:
    train_df , sub_df = target_enc(train_df, cols, target, test_df=sub_df)

In [214]:
train_df.columns

Index(['weekday', 'holiday_flg', 'transaction', 'merchant_id', 'store_id',
       'store_merchant_id', 'active_user', 'GMV', 'category_name',
       'sub_category_name', 'prefecture', 'scan', 'weather', 'humidity',
       'rain', 'Month', 'Day', 'date', 'prefecture_area',
       'store_merchant_id_count', 'sub_category_name_count',
       'transaction_group_win_mean', 'GMV_group_win_mean',
       'active_user_group_win_mean', 'store_merchant_idtransactionmean',
       'store_merchant_idtransactionstd', 'store_merchant_idGMVmean',
       'store_merchant_idGMVstd', 'store_merchant_idactive_usermean',
       'store_merchant_idactive_userstd'],
      dtype='object')

In [215]:
sub_df.columns

Index(['date', 'store_merchant_id', 'transaction', 'weekday', 'merchant_id',
       'store_id', 'category_name', 'sub_category_name', 'prefecture', 'city',
       'village_section', 'scan', 'prefecture_r', 'temperature', 'humidity',
       'rain', 'weather', 'prefecture_area', 'store_merchant_id_count',
       'sub_category_name_count', 'holiday_flg', 'Year', 'Month', 'Day',
       'store_merchant_id_group_win_mean', 'store_merchant_idtransactionmean',
       'store_merchant_idtransactionstd', 'store_merchant_idGMVmean',
       'store_merchant_idGMVstd', 'store_merchant_idactive_usermean',
       'store_merchant_idactive_userstd'],
      dtype='object')

In [218]:
sub_df.head()

,date,store_merchant_id,transaction,weekday,merchant_id,store_id,category_name,sub_category_name,prefecture,city,...,Year,Month,Day,store_merchant_id_group_win_mean,store_merchant_idtransactionmean,store_merchant_idtransactionstd,store_merchant_idGMVmean,store_merchant_idGMVstd,store_merchant_idactive_usermean,store_merchant_idactive_userstd
0,2019-09-30,0,NaN,5,350,0,2,11,1,566,...,2019,9,30,11.25,12.620690,5.038522,35501.758621,33669.271302,11.655172,4.669627
1,2019-09-30,1,NaN,5,3,1,2,17,10,50,...,2019,9,30,0.75,0.344828,0.669534,5272.379310,19187.950930,0.310345,0.541390
2,2019-09-30,2,NaN,5,65,2,2,4,9,167,...,2019,9,30,9.25,7.931034,2.737713,5278.655172,2271.290276,7.517241,2.707406
3,2019-09-30,3,NaN,5,564,3,2,26,5,237,...,2019,9,30,0.25,0.482759,0.687682,67.586207,94.515878,0.413793,0.568032
4,2019-09-30,4,NaN,5,179,4,2,24,9,683,...,2019,9,30,64.75,53.103448,17.879247,120139.482759,56617.591665,49.482759,15.527903


In [220]:
sub_df = sub_df.rename(columns = {'store_merchant_id_group_win_mean':'transaction_group_win_mean'})

In [ ]:
sub_df

In [229]:
columns = [  'weekday',
        'category_name', 'sub_category_name', 'prefecture',  'scan', 'humidity',
       'rain', 'weather', 'prefecture_area',
       'sub_category_name_count', 'holiday_flg', 'Month', 'Day',
       'transaction_group_win_mean', 'store_merchant_idtransactionmean',
       'store_merchant_idtransactionstd', 'store_merchant_idGMVmean',
       'store_merchant_idGMVstd', 'store_merchant_idactive_usermean',
       'store_merchant_idactive_userstd','transaction']

In [230]:
train_df[columns]

,weekday,category_name,sub_category_name,prefecture,scan,humidity,rain,weather,prefecture_area,sub_category_name_count,...,Month,Day,transaction_group_win_mean,store_merchant_idtransactionmean,store_merchant_idtransactionstd,store_merchant_idGMVmean,store_merchant_idGMVstd,store_merchant_idactive_usermean,store_merchant_idactive_userstd,transaction
0,0,2,11,10,0,245,0,5,2,0.121553,...,9,1,NaN,16.517241,5.138237,29306.310345,14493.701270,16.068966,5.098536,13
1,0,2,4,14,0,188,0,3,11,0.606838,...,9,1,NaN,8.172414,3.094569,4436.896552,1915.923357,7.379310,2.846223,5
2,0,2,4,2,0,34,0,3,14,0.606838,...,9,1,NaN,2.137931,1.274030,1344.034483,972.219019,2.137931,1.274030,5
3,0,2,4,1,0,249,0,5,12,0.606838,...,9,1,NaN,45.137931,12.580382,19241.965517,9434.956088,42.827586,12.279684,23
4,0,2,4,10,0,245,0,5,2,0.606838,...,9,1,NaN,18.551724,5.481047,11332.793103,3786.934686,17.724138,5.236524,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503761,0,2,4,12,0,323,0,5,6,0.606838,...,9,29,32.50,29.517241,9.379387,17321.827586,10278.734636,27.827586,9.106783,19
1503762,0,2,4,5,0,121,0,5,0,0.606838,...,9,29,15.25,13.862069,3.562946,7245.000000,2811.766298,13.172414,3.284911,19
1503763,0,2,4,8,0,123,0,5,13,0.606838,...,9,29,20.50,18.172414,4.848188,11159.827586,4183.848289,17.413793,4.444236,19
1503764,0,2,4,12,0,323,0,5,6,0.606838,...,9,29,20.50,17.241379,5.376239,13199.827586,4135.845760,15.379310,5.073839,19


In [221]:
sub_df.to_pickle("./sub_df_9_20_2.pickle")

In [228]:
sub_df[columns]

,weekday,category_name,sub_category_name,prefecture,scan,humidity,rain,weather,prefecture_area,store_merchant_id_count,...,Month,Day,transaction_group_win_mean,store_merchant_idtransactionmean,store_merchant_idtransactionstd,store_merchant_idGMVmean,store_merchant_idGMVstd,store_merchant_idactive_usermean,store_merchant_idactive_userstd,transaction
0,5,2,11,1,1,6,0,2,12,NaN,...,9,30,11.25,12.620690,5.038522,35501.758621,33669.271302,11.655172,4.669627,NaN
1,5,2,17,10,1,7,0,1,2,NaN,...,9,30,0.75,0.344828,0.669534,5272.379310,19187.950930,0.310345,0.541390,NaN
2,5,2,4,9,0,5,2,1,8,0.000019,...,9,30,9.25,7.931034,2.737713,5278.655172,2271.290276,7.517241,2.707406,NaN
3,5,2,26,5,1,0,0,3,0,0.000019,...,9,30,0.25,0.482759,0.687682,67.586207,94.515878,0.413793,0.568032,NaN
4,5,2,24,9,1,5,2,1,8,0.000019,...,9,30,64.75,53.103448,17.879247,120139.482759,56617.591665,49.482759,15.527903,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51849,5,4,27,11,0,8,0,1,3,NaN,...,9,30,0.50,0.241379,0.510964,1567.344828,3209.922465,0.241379,0.510964,NaN
51850,5,1,5,6,1,10,0,0,10,NaN,...,9,30,0.25,0.310345,0.603765,467.241379,987.503695,0.310345,0.603765,NaN
51851,5,2,4,4,0,4,0,1,4,NaN,...,9,30,5.25,5.965517,2.485425,3856.206897,2143.868289,5.827586,2.391487,NaN
51852,5,2,4,10,0,7,0,1,2,NaN,...,9,30,20.75,23.620690,6.079320,12127.172414,4386.327557,22.310345,5.676414,NaN


In [231]:
sub_df[columns].to_pickle("./sub_df_droped.pickle")
train_df[columns].to_pickle("./train_df_droped.pickle")


In [232]:
!mkdir archive

In [236]:
import pickle
sub_df.to_pickle('archive/sub_df')
train_df.to_pickle('archive/train_df')

with open('archive/columns', 'wb') as f:
    pickle.dump(columns, f)

In [11]:
september_encoded_train_df2 = count_encoding(september_encoded_train_df, 'prefecture', )
september_encoded_train_df2 =  count_encoding(september_encoded_train_df2, 'sub_category_name', )

september_encoded_train_df2 = target_enc(september_encoded_train_df2, ['store_merchant_id'], 'transaction')
september_encoded_train_df2 = target_enc(september_encoded_train_df2, ['store_merchant_id'], 'GMV')
september_encoded_train_df2 = target_enc(september_encoded_train_df2, ['store_merchant_id'], 'active_user')

In [13]:
import datetime as dt
# train
train_end = september_encoded_train_df.date.searchsorted(dt.datetime(2019, 9, 27))
df_for_model = september_encoded_train_df.drop('date', 1)
df_for_model = df_for_model.drop('store_id', 1)
df_for_model = df_for_model.drop('merchant_id', 1)

df_for_model.iloc[:train_end].to_csv('train_data/train.csv', index=False)
df_for_model.iloc[train_end:].to_csv('train_data/val.csv', index=False)

In [14]:
import train
import model

model_instance = model.lightGBM('./train_data')
model_instance.train(nround=700)
model_instance.get_importance().sort_values()

[1]	valid_0's rmse: 22.6033
Training until validation scores don't improve for 5 rounds
[2]	valid_0's rmse: 20.3556
[3]	valid_0's rmse: 18.34
[4]	valid_0's rmse: 16.5261
[5]	valid_0's rmse: 14.897
[6]	valid_0's rmse: 13.437
[7]	valid_0's rmse: 12.1296
[8]	valid_0's rmse: 10.9565
[9]	valid_0's rmse: 9.90527
[10]	valid_0's rmse: 8.9697
[11]	valid_0's rmse: 8.13051
[12]	valid_0's rmse: 7.37976
[13]	valid_0's rmse: 6.72106
[14]	valid_0's rmse: 6.12772
[15]	valid_0's rmse: 5.6022
[16]	valid_0's rmse: 5.13645
[17]	valid_0's rmse: 4.72346
[18]	valid_0's rmse: 4.35569
[19]	valid_0's rmse: 4.03033
[20]	valid_0's rmse: 3.74364
[21]	valid_0's rmse: 3.49499
[22]	valid_0's rmse: 3.27868
[23]	valid_0's rmse: 3.0871
[24]	valid_0's rmse: 2.92936
[25]	valid_0's rmse: 2.77932
[26]	valid_0's rmse: 2.65622
[27]	valid_0's rmse: 2.54831
[28]	valid_0's rmse: 2.45808
[29]	valid_0's rmse: 2.37998
[30]	valid_0's rmse: 2.30614
[31]	valid_0's rmse: 2.24441
[32]	valid_0's rmse: 2.20499
[33]	valid_0's rmse: 2.16531

,importance
weekday,62
holiday_flg,48
store_merchant_id,413
active_user,1431
GMV,538
category_name,42
sub_category_name,117
prefecture,61
scan,21
weather,35
